In [1]:
!java -version

openjdk version "22.0.1" 2024-04-16
OpenJDK Runtime Environment (build 22.0.1+8-16)
OpenJDK 64-Bit Server VM (build 22.0.1+8-16, mixed mode, sharing)


### This downloads and adds OpenJDK Java stuff to Java directory

In [ ]:
!curl "https://download.java.net/java/GA/jdk22.0.1/c7ec1332f7bb44aeba2eb341ae18aca4/8/GPL/openjdk-22.0.1_macos-x64_bin.tar.gz" | tar zxf -

### This is where you should put it

In [ ]:
!mv jdk-22.0.1.jdk /Library/Java/JavaVirtualMachines/

In [2]:
!pip install -q tabula-py

In [3]:
!pip install jpype1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 583.5/583.5 kB 8.2 MB/s  0:00:00


In [1]:
import tabula
import pandas as pd

### This tells your Python profile where to find the Java JDK contents

In [2]:
!echo export "JAVA_HOME=/Library/Java/JavaVirtualMachines/jdk-22.0.1.jdk/Contents/Home" >> ~/.zshrc

In [3]:
pdf_path = "2024-covid-19-block-grant-reports-7-26-24-a11y.pdf"

In [13]:
df_block = tabula.read_pdf(pdf_path, multiple_tables=False, pages="27-47", pandas_options={'header': None})
df_block = pd.concat(df_block, axis=1, ignore_index=True)
# Optional cleanup
df_block = df_block.reset_index(drop=True)

In [4]:
# --- Step 1: Extract all tables ---
dfs = tabula.read_pdf(
    pdf_path,
    pages="27-47",
    multiple_tables=True,
    pandas_options={'header': 0, 'dtype': str}
)

# Flatten if multiple_tables=True returns nested lists
dfs_flat = []
for item in dfs:
    if isinstance(item, list):
        dfs_flat.extend(item)
    else:
        dfs_flat.append(item)

# Drop empties
dfs_flat = [df for df in dfs_flat if df is not None and not df.empty]

# --- Step 2: Promote first row as header if not already ---
cleaned = []
for df in dfs_flat:
    if df.columns[0] == 0 or df.columns.dtype != object:
        # Sometimes header wasn't picked up correctly
        df.columns = df.iloc[0]
        df = df.iloc[1:].reset_index(drop=True)
    cleaned.append(df)

# --- Step 3: Group by header signature ---
# Define a signature for each table’s header to detect sameness
def header_signature(df):
    return tuple(str(c).strip().lower() for c in df.columns)

grouped = []
current_group = [cleaned[0]]
current_sig = header_signature(cleaned[0])

for df in cleaned[1:]:
    sig = header_signature(df)
    if sig == current_sig:
        # same header → stack vertically
        current_group.append(df)
    else:
        # new header → finish previous vertical stack
        grouped.append(pd.concat(current_group, ignore_index=True))
        current_group = [df]
        current_sig = sig

# append the final group
grouped.append(pd.concat(current_group, ignore_index=True))

# --- Step 4: Concatenate horizontally (side by side) across header groups ---
final_df = pd.concat(grouped, axis=1)

# optional cleanup
final_df = final_df.reset_index(drop=True)


In [9]:
final_df.columns.values[0] = "block_use"

In [13]:
final_df = final_df.drop(columns=["Category", "Subcategory", "..."], errors="ignore")

In [22]:
df = final_df[final_df['block_use'].str.contains('Discharging unpaid fees|Total Expended|Total Expenditures and Encumbrance', case=False, na=False)]

In [25]:
# Step 1: Set the first column as the new header row
df_t = df.set_index(df.columns[0]).T

# Step 2: Reset the index to turn old column headers into a column
df_t = df_t.reset_index().rename(columns={'index': 'district'})

df_t = df.set_index('block_use').T.reset_index().rename(columns={'index':'district'})

df_t

block_use,district,Discharging unpaid fees,Total Expended,Total Expenditures and Encumbrances
0,Statewide Total,"$20,199,178","$207,812,480","$403,306,926"
1,Allan Hancock Joint,-,"279,267","5,151,904"
2,Antelope Valley,"2,331,678","2,667,197","5,825,633"
3,Barstow,"250,969","1,452,472","1,518,489"
4,Butte-Glenn,-,"6,444,216","6,857,648"
...,...,...,...,...
82,West Hills,-,"785,296","1,589,297"
83,West Kern,-,"731,095","731,095"
84,West Valley-Mission,"1,967,208","3,011,994","6,802,861"
85,Yosemite,-,"1,411,114","9,554,739"


In [17]:
df.to_csv(
    "block_grant_fee_discharges.csv", index=False)

In [18]:
df

,district,col_1,col_2
0,Discharging unpaid fees,Total Expended,Total Expenditures and Encumbrances
1,"$20,199,178","$207,812,480","$403,306,926"
2,-,"279,267","5,151,904"
3,"2,331,678","2,667,197","5,825,633"
4,"250,969","1,452,472","1,518,489"
...,...,...,...
83,-,"785,296","1,589,297"
84,-,"731,095","731,095"
85,"1,967,208","3,011,994","6,802,861"
86,-,"1,411,114","9,554,739"


In [ ]:
df.to_csv(
    "d_ftb_offset_2023.csv", index=False)

In [31]:
[j for i in range(7, df.shape[1], 6) for j in (i, i+1) if j < df.shape[1]]

[7, 8, 13, 14, 19, 20, 25, 26, 31, 32, 37, 38, 43, 44, 49, 50, 55, 56]

In [20]:
df = final_df.drop(final_df.columns[[i for i in range(7, final_df.shape[1], 6) for i in (i, i+1) if i < final_df.shape[1]]], axis=1)

In [7]:
def drop_col_pattern(df, start=7, step=6):
    """
    Drop the 8th & 9th columns (0-based 7,8), then every 6th & 7th after.
    Works regardless of column labels (uses .iloc positions only).
    """
    drop_pos = []
    i = start
    while i < final_df.shape[1]:
        drop_pos.extend([i, i + 1])
        i += step
    drop_pos = [p for p in drop_pos if p < final_df.shape[1]]

    # Keep all columns NOT in drop_pos, by position
    keep_pos = [i for i in range(df.shape[1]) if i not in drop_pos]
    return final_df.iloc[:, keep_pos].copy()

# Usage
df = drop_col_pattern(final_df)  # start=7 (8th col), step=6 by default
df

,Category,Subcategory,Statewide Total,Allan Hancock Joint,Antelope Valley,Barstow,Butte-Glenn,Cabrillo,Calbright,Cerritos,...,Solano County,Sonoma\rCounty,Subcategory,Southwestern,State Center,Ventura County,Category,Subcategory,West Kern,West Valley-Mission
0,Basic Needs,Food Programs,"$7,295,657","30,583",-,-,-,"117,760",-,-,...,-,-,Food Programs,-,"40,281","33,329",Basic Needs,Food Programs,-,"29,156"
1,Basic Needs,Housing Support,"$869,834",-,-,-,-,-,-,-,...,-,-,Housing Support,-,"722,910",-,Basic Needs,Housing Support,-,-
2,Direct Emergency Grants to Students,Financial Aid Grants,"$12,519,484",-,-,-,-,"396,254",-,-,...,-,-,Financial Aid Grants,"477,568",-,"2,000",Direct Emergency Grants to Students,Financial Aid Grants,-,"46,000"
3,Direct Emergency Grants to Students,Learning Tools,"$402,918",-,-,-,-,-,-,-,...,-,-,Learning Tools,-,-,-,Direct Emergency Grants to Students,Learning Tools,-,-
4,Discharging unpaid fees,Unpaid fees due or owed by student to communit...,"$20,199,178",-,"2,331,678","250,969",-,"1,394,784",-,-,...,"850,000","2,938,980",Unpaid fees due or owed by student to communit...,-,276,-,Discharging unpaid fees,Unpaid fees due or owed by student to communit...,-,"1,967,208"
5,Facilitate Distance Learning,"Grants to faculty to develop online, accelerat...","$367,138",-,-,-,-,"9,403",-,-,...,-,-,"Grants to faculty to develop online, accelerat...",-,-,"18,660",Facilitate Distance Learning,"Grants to faculty to develop online, accelerat...",-,-
6,Investments to Close the Digital Divide,Information Technology Infrastructure,"$15,580,300",-,"146,823","39,562",-,-,-,-,...,-,-,Information Technology Infrastructure,"318,809","95,223","160,663",Investments to Close the Digital Divide,Information Technology Infrastructure,"731,095","127,578"
7,Investments to Close the Digital Divide,Facilitate Student Access to Technology,"$8,203,244",-,-,"198,428","39,508","63,171",-,-,...,-,-,Facilitate Student Access to Technology,-,"249,886","317,322",Investments to Close the Digital Divide,Facilitate Student Access to Technology,-,"203,832"
8,Investments to Close the Digital Divide,Software and Hardware purchases that enable em...,"$6,902,336",-,-,"58,562","8,294","29,616",-,-,...,"3,668,244","3,272,748",Software and Hardware purchases that enable em...,-,"64,533","26,265",Investments to Close the Digital Divide,Software and Hardware purchases that enable em...,-,"205,657"
9,Mental Health Services and Support,Mental Health Services and Support,"$1,976,789",-,-,"11,047","18,129",-,-,"130,931",...,-,"9,750,332",Mental Health Services and Support,-,"27,474","38,552",Mental Health Services and Support,Mental Health Services and Support,-,-


In [17]:
# Build the column indices to drop
drop_indices = []
start = 7  # 8th column (0-based)
while start < df.shape[1]:
    drop_indices.extend([start, start + 1])
    start += 6  # move 6 columns forward from the first in each pair

# Drop those columns safely (ignore if out of range)
drop_indices = [i for i in drop_indices if i < df.shape[1]]

# Drop by index position
df = final_df.drop(final_df.columns[drop_indices], axis=1)

In [18]:
df

,Category,Subcategory,Statewide Total,Allan Hancock Joint,Antelope Valley,Barstow,Butte-Glenn,Category,Subcategory,Cabrillo,...,State Center,Ventura County,Victor Valley,West Hills,Category,Subcategory,West Kern,West Valley-Mission,Yosemite,Yuba
0,Basic Needs,Food Programs,"$7,295,657","30,583",-,-,-,Basic Needs,Food Programs,"117,760",...,"40,281","33,329",-,-,Basic Needs,Food Programs,-,"29,156","75,000",-
1,Basic Needs,Housing Support,"$869,834",-,-,-,-,Basic Needs,Housing Support,-,...,"722,910",-,-,-,Basic Needs,Housing Support,-,-,-,-
2,Direct Emergency Grants to Students,Financial Aid Grants,"$12,519,484",-,-,-,-,Direct Emergency Grants to Students,Financial Aid Grants,"396,254",...,-,"2,000","24,376","372,137",Direct Emergency Grants to Students,Financial Aid Grants,-,"46,000",-,"1,362"
3,Direct Emergency Grants to Students,Learning Tools,"$402,918",-,-,-,-,Direct Emergency Grants to Students,Learning Tools,-,...,-,-,-,-,Direct Emergency Grants to Students,Learning Tools,-,-,-,-
4,Discharging unpaid fees,Unpaid fees due or owed by student to communit...,"$20,199,178",-,"2,331,678","250,969",-,Discharging unpaid fees,Unpaid fees due or owed by student to communit...,"1,394,784",...,276,-,"161,292",-,Discharging unpaid fees,Unpaid fees due or owed by student to communit...,-,"1,967,208",-,-
5,Facilitate Distance Learning,"Grants to faculty to develop online, accelerat...","$367,138",-,-,-,-,Facilitate Distance Learning,"Grants to faculty to develop online, accelerat...","9,403",...,-,"18,660",-,-,Facilitate Distance Learning,"Grants to faculty to develop online, accelerat...",-,-,-,-
6,Investments to Close the Digital Divide,Information Technology Infrastructure,"$15,580,300",-,"146,823","39,562",-,Investments to Close the Digital Divide,Information Technology Infrastructure,-,...,"95,223","160,663",-,-,Investments to Close the Digital Divide,Information Technology Infrastructure,"731,095","127,578",-,"312,825"
7,Investments to Close the Digital Divide,Facilitate Student Access to Technology,"$8,203,244",-,-,"198,428","39,508",Investments to Close the Digital Divide,Facilitate Student Access to Technology,"63,171",...,"249,886","317,322",-,"351,212",Investments to Close the Digital Divide,Facilitate Student Access to Technology,-,"203,832","301,619","63,931"
8,Investments to Close the Digital Divide,Software and Hardware purchases that enable em...,"$6,902,336",-,-,"58,562","8,294",Investments to Close the Digital Divide,Software and Hardware purchases that enable em...,"29,616",...,"64,533","26,265","332,343","7,320",Investments to Close the Digital Divide,Software and Hardware purchases that enable em...,-,"205,657","63,486","47,766"
9,Mental Health Services and Support,Mental Health Services and Support,"$1,976,789",-,-,"11,047","18,129",Mental Health Services and Support,Mental Health Services and Support,-,...,"27,474","38,552",-,-,Mental Health Services and Support,Mental Health Services and Support,-,-,"50,000",-


In [ ]:
df_block = pd.concat(df_block, ignore_index=True)
df_block = df_block.iloc[30:]
df_block = df_block.reset_index(drop=True)
df_block = df_block.drop(0)
df_block = df_block.drop(0, axis=1)

In [14]:
df_block

,0,1,2,3,4,5,6
0,Category,Subcategory,Statewide Total,Allan Hancock Joint,Antelope Valley,Barstow,Butte-Glenn
1,Basic Needs,Food Programs,"$7,295,657","30,583",-,-,-
2,Basic Needs,Housing Support,"$869,834",-,-,-,-
3,Direct Emergency Grants to Students,Financial Aid Grants,"$12,519,484",-,-,-,-
4,Direct Emergency Grants to Students,Learning Tools,"$402,918",-,-,-,-
...,...,...,...,...,...,...,...
475,Category,Subcategory,West Kern,West Valley-Mission,Yosemite,Yuba,NaN
476,Total Expended,-,"731,095","3,011,994","1,411,114","431,720",NaN
477,Total Encumbered,-,-,"3,790,867","8,143,625",-,NaN
478,Total Expenditures and Encumbrances,-,"731,095","6,802,861","9,554,739","431,720",NaN


In [ ]:
df_block = df_block.rename(columns=
    {1: 'agencytype', 2:'name', 3:'pit_count', 4:'pit_amount',
     5:'lottery_count', 6:'lottery_amount', 7:'property_count',
     8:'property_amount', 9: 'all_count', 10: 'all_amount'})
df_2023

In [6]:
pdf_path = "2024-covid-19-block-grant-reports-7-26-24-a11y.pdf"

In [33]:
df_2023 = tabula.read_pdf(pdf_path, multiple_tables=False, pages='19-42', pandas_options={'header': None})
df_2023 = pd.concat(df_2023, ignore_index=True)
df_2023 = df_2023.iloc[30:]
df_2023 = df_2023.reset_index(drop=True)
df_2023 = df_2023.drop(0)
df_2023 = df_2023.drop(0, axis=1)
df_2023 = df_2023.rename(columns=
    {1: 'agencytype', 2:'name', 3:'pit_count', 4:'pit_amount',
     5:'lottery_count', 6:'lottery_amount', 7:'property_count',
     8:'property_amount', 9: 'all_count', 10: 'all_amount'})
df_2023

,agencytype,name,pit_count,pit_amount,lottery_count,lottery_amount,property_count,property_amount,all_count,all_amount
1,NaN,CITY OF PARLIER,32,"$6,152.00",NaN,NaN,NaN,NaN,32,"$6,152.00"
2,NaN,CITY OF EXETER,7,$679.00,NaN,NaN,NaN,NaN,7,$679.00
3,NaN,CITY OF SHASTA LAKE,7,$721.00,NaN,NaN,NaN,NaN,7,$721.00
4,NaN,CITY OF GARDEN GROVE,761,"$141,768.19",9,"$2,386.14",8,$520.70,778,"$144,675.03"
5,NaN,CITY OF REDDING,7,"$1,211.46",NaN,NaN,NaN,NaN,7,"$1,211.46"
...,...,...,...,...,...,...,...,...,...,...
622,NaN,SUPERIOR COURT OF SAN FRANCISCO,"1,844","$549,501.81",25,"$10,923.75",54,"$7,347.93","1,923","$567,773.49"
623,NaN,SAN DIEGO SUPERIOR COURT,"4,251","$1,255,279.80",47,"$20,325.07",90,"$16,820.02","4,388","$1,292,424.89"
624,Z - OTHER Total,NaN,"6,712","$2,003,259.91",80,"$37,284.57",162,"$27,917.47","6,954","$2,068,461.95"
625,NaN,NaN,"822,680","$345,673,915.94","8,865","$7,198,070.02","24,302","$7,267,478.12","855,847","$360,139,464.08"


In [34]:
df_2023.to_csv(
    "d_ftb_offset_2023.csv", index=False)

In [7]:
dfconcat_cols1to3 = pd.concat(df_cols1to3)

dfconcat_cols1to3['row'] = dfconcat_cols1to3.reset_index().index

In [8]:
df_cols4to6 = tabula.read_pdf(pdf_path, multiple_tables=True, pages=[16,20,26,27],
    pandas_options={'header': None})

dfconcat_cols4to6 = pd.concat(df_cols4to6)

dfconcat_cols4to6['row'] = dfconcat_cols4to6.reset_index().index

In [9]:
df_cols7to9 = tabula.read_pdf(pdf_path, multiple_tables=True, pages=[17,21,28,29],
    pandas_options={'header': None})

dfconcat_cols7to9 = pd.concat(df_cols7to9)

dfconcat_cols7to9['row'] = dfconcat_cols7to9.reset_index().index

In [10]:
df_ftb_merge_cols1to3_7to9 = dfconcat_cols1to3.merge(dfconcat_cols7to9,on='row')

In [11]:
df_ftb_merge_cols1to3_7to9 = df_ftb_merge_cols1to3_7to9.rename(columns=
    {'0_x': 'agency', '1_x': 'pit_count', '2_x': 'pit_amount', 
     '1_y': 'all_count', '2_y': 'all_amount'})

In [12]:
df_ftb_merge_cols1to3_7to9.dropna(subset=['agency'], inplace=True)

In [13]:
df_ftb_merge_cols1to3_7to9 = df_ftb_merge_cols1to3_7to9.drop(columns=['row', '0_y'])

In [14]:
pd.set_option('display.max_rows', None)

In [15]:
df_ftb_merge_cols1to3_7to9

,agency,pit_count,pit_amount,all_count,all_amount
0,CALF STATE POLYTECHNIC UNIVERSITY,534,"$140,825.24",546,"$143,027.04"
1,CALIFORNIA STATE UNIV NORTHRIDGE,"1,056","$328,633.46","1,099","$341,169.88"
2,"UNIVERSITY OF CALIFORNIA, OP","3,139","$1,401,548.97","3,218","$1,412,893.93"
3,CSU SACRAMENTO,665,"$176,150.92",685,"$177,456.73"
4,UCLA,381,"$50,701.63",388,"$51,390.12"
5,"CSU, BAKERSFIELD",608,"$167,703.71",630,"$172,614.60"
6,"UNIVERSITY OF CALIFORNIA, RIVERSIDE",674,"$263,721.21",700,"$269,411.04"
7,UC SANTA BARBARA,299,"$43,273.59",301,"$43,414.02"
8,CSU MARITIME ACADEMY,29,"$21,760.83",31,"$22,182.28"
9,FRESNO STATE,75,"$9,173.41",77,"$9,430.41"


In [16]:
df_ftb_merge_cols1to3_7to9.to_csv(
    "d_ftb_offset_2019_pra.csv", index=False)